# Conceitos

# Exercício Proposto

Modelo Vetorial

Dada a coleção cedida pelo docente, faça um programa Python que:

1. Retira stopwords, dígitos, espaços adicionais, hífens, pontuação, tokeniza cada texto (documento).

2. Gerar o vocabulário dos documentos da coleção (palavras simples).

3. Para cada documento da coleção gerar o vetor com pesos tf-idf para cada termo do documento, existente no vocabulário. Esse vetor contém todos os termos do vocabulário.

4. Ler a consulta do teclado como uma lista de termos, gerar o vetor de pesos tf-idf da consulta, de acordo com o vocabulário.

5. Ranquear em ordem decrescente (maior para o menor) os documentos, pontuados no cálculo de similaridade do vetor de documento com o vetor de consulta.

6. Observe que todos os vetores têm o mesmo tamanho, e a posição de cada elemento de um vetor corresponde a um termo do vocabulário com seu peso tf-idf para um documento ou a consulta.

![fluxo](https://i.postimg.cc/bJfWt8TQ/Untitled-2024-06-18-1502.png)

In [ ]:
!pip install Unidecode
!pip install --user -U nltk
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.3 MB/s eta 0:00:00 0:00:01


In [ ]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 37.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 27.7 MB/s eta 0:00:0000:0100:01


In [ ]:
# Importing

In [2]:
from google.colab import files
import pandas as pd
import time
import nltk
import re
import os
import glob

KeyboardInterrupt: 

In [ ]:
# Loading Data

In [ ]:
# Subir aqui para o arquivos do colab

def carregar_arquivos(diretorio: str) -> list:
    """
    Lê todos os arquivos .txt do diretório do corpus e retorna uma lista com o conteúdo de cada arquivo.
    diretorio (str): É o caminho para o diretório contendo os arquivos .txt.
    list: Uma lista onde cada elemento é o conteúdo de um arquivo .txt.
    """

    arquivos_conteudo = []

    if os.path.isdir(diretorio):
        # Procurar todos os arquivos .txt no diretório
        for arquivo in glob.glob(os.path.join(diretorio, '*.txt')):
            try:
                with open(arquivo, 'r', encoding='utf-8') as txt:
                    arquivos_conteudo.append(txt.read().lower())
            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {e}")
    else:
        print(f"O diretório {diretorio} não existe.")

    return arquivos_conteudo

In [ ]:
# Caminho dos repositórios
caminho_EdFisica_txt = "./data/EdFisica_txt/"
caminho_Geografia_txt = "./data/Geografia_txt/"
caminho_Historia_txt = "./data/Historia_txt/"
caminho_Linguistica_txt = "./data/Linguistica_txt/"

# Carga de dados
raw_edfisica = carregar_arquivos(caminho_EdFisica_txt)
raw_geografia = carregar_arquivos(caminho_Geografia_txt)
raw_historia = carregar_arquivos(caminho_Historia_txt)
raw_linguistica = carregar_arquivos(caminho_Linguistica_txt)

1. Retira stopwords, dígitos, espaços adicionais, hífens, pontuação, tokeniza cada texto (documento).

In [1]:
nltk.download('stopwords')
print(stopwords.words('portuguese'))

NameError: name 'nltk' is not defined

In [ ]:
from concurrent.futures import ThreadPoolExecutor  # Para execução assíncrona e paralela usando um pool de threads
from nltk.corpus import stopwords  # Lista de palavras comuns a serem ignoradas em NLP
from string import punctuation  # Conjunto de caracteres de pontuação padrão
from unidecode import unidecode  # Converte caracteres Unicode para ASCII simples, removendo acentos
from collections import Counter  # Conta a frequência de elementos em uma lista ou texto
from tqdm import tqdm  # Adiciona uma barra de progresso visual para loops e iterações

In [ ]:
def tokenize_texto(text, stop_words, nao_alphanumerico, digitos, mult_espacos):

    cl = nao_alphanumerico.sub('', text)
    cl = digitos.sub('', cl)
    cl = mult_espacos.sub(' ', cl)
    cl = cl.lower().strip()
    cl = unidecode(cl)
    palavras = cl.split()

    palavras_filtradas = [palavra for palavra in palavras if palavra not in stop_words]

    return palavras_filtradas

def processamento(texts):

    """
    Essa função compilar as expressões regulares e seta as váriaveis,
    onde será aplicado no processamento
    dos textos utilizando a função tokenize_texto
    usando o ThreadPoolExecutor, e posteriormente
    gerar o vocabulário a partir dos textos processados.

    """

    # Compilando expressões regulares para reutilização
    nao_alphanumerico = re.compile(r'[^\w\s]')
    digitos = re.compile(r'\d')
    mult_espacos = re.compile(r'\s+')

    # Carregar stopwords para português, inglês e espanhol, adicionando stopwords customizadas
    adicional_stopwords = ['v', 'ja', 'b']
    stop_words = set(stopwords.words('portuguese') +
                     stopwords.words('english') +
                     stopwords.words('spanish') +
                     adicional_stopwords).union(set(punctuation))

    # Medir o tempo de início
    inicio = time.time()

    # Usando ThreadPoolExecutor para paralelizar o processamento de textos e tokenização
    with ThreadPoolExecutor(max_workers=4) as executor:
        resultados = list(tqdm(executor.map(lambda text:
        tokenize_texto(text, stop_words, nao_alphanumerico, digitos, mult_espacos), texts),
        total=len(texts), ncols=100))

    # O texts é a quantidade de arquivos dentro do diretório de cada matéria

    # Construir o vocabulário
    # vocabulario = []

    # for tokens in resultados:
    #     vocabulario.extend(tokens)

    vocabulario = set()

    for tokens in resultados:
        vocabulario.update(tokens)

    # Medir o tempo de fim
    fim = time.time()
    tempo = fim - inicio

    print(f"Tempo total de processamento: {tempo:.2f} segundos")
    print(f"Total de palavras únicas no vocabulário: {len(vocabulario)}")

    return vocabulario

In [ ]:
# Dados limpos e adaptados
bronze_EdFisica_txt = processamento(raw_edfisica)
bronze_Geografia_txt = processamento(raw_geografia)
bronze_Historia_txt = processamento(raw_historia)
bronze_Linguistica_txt = processamento(raw_linguistica)

100%|████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 22344.09it/s]


Tempo total de processamento: 0.19 segundos
Total de palavras únicas no vocabulário: 13290


100%|██████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 935.63it/s]


Tempo total de processamento: 0.25 segundos
Total de palavras únicas no vocabulário: 14389


100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 122461.43it/s]


Tempo total de processamento: 0.28 segundos
Total de palavras únicas no vocabulário: 19278


100%|█████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 8938.31it/s]

Tempo total de processamento: 0.33 segundos
Total de palavras únicas no vocabulário: 19706


In [ ]:
print(bronze_EdFisica_txt)

2. Gerar o vocabulário dos documentos da coleção (palavras simples).

- GERADO NA FUNÇÃO 'processamento', onde pode ser verificado na célular anterior, o código de gerar o vocabulário a partir dos textos

3. Para cada documento da coleção gerar o vetor com pesos tf-idf para cada termo do documento, existente no vocabulário. Esse vetor contém todos os termos do vocabulário.

In [ ]:
# Cria conjuntos para cada arquivo e faz a união
setDicionarios = set(bronze_EdFisica_txt).union(
    set(bronze_Geografia_txt),
    set(bronze_Historia_txt),
    set(bronze_Linguistica_txt)
)

# Conta o total de palavras únicas
total_palavras_unicas = len(setDicionarios)

print("Total de palavras únicas:", total_palavras_unicas)

Total de palavras únicas: 43324


In [ ]:
print(setDicionarios)

{'regimes', 'gen', 'edufu', 'modulacao', 'buscado', 'respaldando', 'quintou', 'neira', 'estadia', 'bichinha', 'alrededor', 'dezessete', 'participativas', 'limitado', 'imaginados', 'tapar', 'prolongamen', 'permitidos', 'comprovar', 'concourse', 'poeticas', 'eac', 'linterieur', 'endogeno', 'wide', 'resul', 'twentieth', 'pesquisador', 'decs', 'lencol', 'segue', 'existen', 'fischer', 'sartorelli', 'casalegno', 'desesperado', 'gemina', 'proporciones', 'vigorar', 'aplicaram', 'valorizala', 'provisoes', 'raca', 'trabajar', 'berger', 'angelina', 'ppgeo', 'requalificar', 'empenhado', 'acoplamientos', 'desafio', 'conjectural', 'beholder', 'timetrial', 'relativiza', 'palmares', 'gupta', 'medieval', 'experien', 'interpretativos', 'mudado', 'supportive', 'gramatica', 'nainr', 'conserven', 'rule', 'jiang', 'agravamento', 'jatene', 'kit', 'lancet', 'naonegros', 'apresentarei', 'adquira', 'tradu', 'eur', 'temem', 'nhecimento', 'alienacaoseparacao', 'dr', 'cubos', 'feridas', 'formaram', 'potential', 'c

In [ ]:
# Criar dicionários para cada conjunto de palavras
DictEdFisica = dict.fromkeys(setDicionarios, 0)
DictGeografia = dict.fromkeys(setDicionarios, 0)
DictHistoria = dict.fromkeys(setDicionarios, 0)
DictLinguistica = dict.fromkeys(setDicionarios, 0)

In [ ]:
for word in bronze_EdFisica_txt:
    DictEdFisica[word]+=1

for word in bronze_Geografia_txt:
    DictGeografia[word]+=1

for word in bronze_Historia_txt:
    DictHistoria[word]+=1

for word in bronze_Linguistica_txt:
    DictLinguistica[word]+=1

In [ ]:
print(DictLinguistica)

In [ ]:
def computeTF(wordDict, bronze):
    tfDict = {}
    bowCount = len(bronze)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)

    idfDict = dict.fromkeys(docList[0].keys(), 0)

    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1

    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))

    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [ ]:
tfEdFisica = computeTF(DictEdFisica, bronze_EdFisica_txt)
tfGeografia = computeTF(DictGeografia, bronze_Geografia_txt)
tfHistoria = computeTF(DictHistoria, bronze_Historia_txt)
tfLinguistica = computeTF(DictLinguistica, bronze_Linguistica_txt)

In [ ]:
idfs = computeIDF([DictEdFisica, DictGeografia, DictHistoria, DictLinguistica])

In [ ]:
tfidfEdFisica = computeTFIDF(tfEdFisica, idfs)
tfidfGeografia = computeTFIDF(tfGeografia, idfs)
tfidfHistoria = computeTFIDF(tfHistoria, idfs)
tfidfLinguistica= computeTFIDF(tfLinguistica, idfs)

In [ ]:
import pandas as pd
data = pd.DataFrame([tfidfEdFisica, tfidfGeografia, tfidfHistoria, tfidfLinguistica])
data

,regimes,gen,edufu,modulacao,buscado,respaldando,quintou,neira,estadia,bichinha,...,interferiam,tambara,estuarine,invisibilizado,alirio,magnifico,marciais,learning,destruicao,cartesiana
0,0.000009,0.000045,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000045,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000045,0.000009,0.0,0.000000
1,0.000009,0.000000,0.000000,0.000000,0.000042,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000042,0.000042,0.000000,0.000042,0.000000,0.000000,0.000000,0.0,0.000000
2,0.000006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000031,...,0.000031,0.000000,0.000000,0.000031,0.000000,0.000031,0.000000,0.000006,0.0,0.000031
3,0.000000,0.000000,0.000031,0.000031,0.000000,0.000031,0.000031,0.000031,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000006,0.0,0.000000


4. Ler a consulta do teclado como uma lista de termos, gerar o vetor de pesos tf-idf da consulta, de acordo com o vocabulário.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

"""
cosine_similarity em scikit-learn:

Valor 1: Se a similaridade de cosseno é 1, os vetores são idênticos em direção (o ângulo entre eles é 0°).
Isso indica uma alta similaridade.
Valor 0: Se a similaridade de cosseno é 0, os vetores são ortogonais (o ângulo entre eles é 90°).
Isso indica que não há similaridade.
"""

def processar_consulta(consulta, vocabulario, idfs):
    # Usa a função de tokenização para a consulta
    consulta_tokens = processamento([consulta])

    # Cria um dicionário para a consulta
    dict_consulta = dict.fromkeys(vocabulario, 0)

    for token in consulta_tokens:
        if token in dict_consulta:
            dict_consulta[token] += 1

    # Calcula TF para a consulta
    tf_consulta = computeTF(dict_consulta, consulta_tokens)

    # Calcula TF-IDF para a consulta
    tfidf_consulta = computeTFIDF(tf_consulta, idfs)

    return tfidf_consulta

def calcular_similaridade(vetor_consulta, vetores_documentos, vocabulario):

    # Converte os dicionários em arrays numpy, alinhados pelo vocabulário
    vetor_consulta_array = np.array([vetor_consulta.get(word, 0) for word in vocabulario])
    vetores_documentos_array = np.array([[doc.get(word, 0) for word in vocabulario] for doc in vetores_documentos])

    """
    Os vetores são construídos usando a mesma ordem de termos definida pelo vocabulario.
    Isso significa que, para cada termo no vocabulário, obtemos seu valor TF-IDF no vetor da consulta ou dos documentos.
    Se um termo não estiver presente em um documento ou na consulta, seu valor TF-IDF será 0 (vetor_consulta.get(word, 0) e doc.get(word, 0)).
    """

    # Calcula a similaridade de cosseno
    similaridades = cosine_similarity([vetor_consulta_array], vetores_documentos_array)[0]

    return similaridades

def buscar_documentos(consulta, vocabulario, idfs, documentos):
    # Processa a consulta
    vetor_consulta = processar_consulta(consulta, vocabulario, idfs)

    # Calcula as similaridades
    similaridades = calcular_similaridade(vetor_consulta, documentos, vocabulario)

    # Cria uma lista de tuplas (índice do documento, similaridade)
    resultados = list(enumerate(similaridades))

    # Ordena os resultados por similaridade em ordem decrescente
    resultados_ordenados = sorted(resultados, key=lambda x: x[1], reverse=True)

    return resultados_ordenados

consulta = input("Digite sua consulta: ")

vocabulario = setDicionarios
documentos = [tfidfEdFisica, tfidfGeografia, tfidfHistoria, tfidfLinguistica]
nomes_documentos = ["Educação Física", "Geografia", "História", "Linguística"]

resultados = buscar_documentos(consulta, vocabulario, idfs, documentos)

print("\nResultados da busca:")

for indice, similaridade in resultados:
    if similaridade > 0:
        print(f"{nomes_documentos[indice]}: {similaridade:.4f}")


Digite sua consulta:  difundiram a burguesia sofisticada com o seu produto


100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8507.72it/s]

Tempo total de processamento: 0.00 segundos
Total de palavras únicas no vocabulário: 4

Resultados da busca:
Geografia: 0.0102
Educação Física: 0.0085
História: 0.0016


5. Ranquear em ordem decrescente (maior para o menor) os documentos, pontuados no cálculo de similaridade do vetor de documento com o vetor de consulta.

In [ ]:
# Reposta no exemplo anterior

6. Observe que todos os vetores têm o mesmo tamanho, e a posição de cada elemento de um vetor corresponde a um termo do vocabulário com seu peso tf-idf para um documento ou a consulta.

No código, esse alinhamento é feito no (setDicionarios) seja a referência para todos os vetores TF-IDF. Para cada documento e para a consulta, gerando um vetor onde cada posição representa o peso TF-IDF do termo correspondente no vocabulário.